**Deliverable 1: Retrieve Weather Data**

In [1]:
# Import the dependencies
import pandas as pd
import numpy as np

# Import the requests library
import requests

# Append the "conf" folder (keep the config.py file) to the system path 
import sys
sys.path.append('../conf')

# Import the API key from the config.py file
from config import weather_api_key

# Import the time library and the datetime module from the datetime library 
import time

# Import the citipy module to determine city based on given latitude and longitude
from citipy import citipy

In [2]:
# Create a set of 2000 random latitude and longitude combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=10)
lngs = np.random.uniform(low=-180.000, high=180.000, size=10)
# The zip object packs each pair of lats and lngs having the same index in their respective array into a tuple
lat_lngs = zip(lats, lngs)

In [3]:
# Unpack the lat_lngs zip object into a list
coordinates = list(lat_lngs)

In [4]:
# Create a list for holding the cities
cities = []

# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count.
print("Number of cities selected: " + str(len(cities)))

Number of cities selected: 10


In [5]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [6]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_desc = city_weather["weather"][0]["description"]

        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_weather_desc
                         })

# If an error is experienced, skip the city.
    except:
        print("City '" + city.title() + "' not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | taolanaro
City 'Taolanaro' not found. Skipping...
Processing Record 2 of Set 1 | sioux lookout
Processing Record 3 of Set 1 | mareeba
Processing Record 4 of Set 1 | new norfolk
Processing Record 5 of Set 1 | mataura
Processing Record 6 of Set 1 | udachnyy
Processing Record 7 of Set 1 | ushuaia
Processing Record 8 of Set 1 | amderma
City 'Amderma' not found. Skipping...
Processing Record 9 of Set 1 | yulara
Processing Record 10 of Set 1 | punta arenas
-----------------------------
Data Retrieval Complete      
-----------------------------


In [7]:
# Convert the array of dictionaries to a Pandas DataFrame
city_data_df = pd.DataFrame(city_data)
#city_data_df.head(10)

In [8]:
# Reorder the columns of the city_data_df DataFrame
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]

city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Sioux Lookout,CA,50.1001,-91.9170,10.47,61,40,9.22,scattered clouds
1,Mareeba,AU,-17.0000,145.4333,74.44,96,100,1.16,overcast clouds
2,New Norfolk,AU,-42.7826,147.0587,52.75,68,23,1.01,few clouds
3,Mataura,NZ,-46.1927,168.8643,56.39,76,100,4.29,overcast clouds
4,Udachnyy,RU,66.4167,112.4000,-8.95,99,99,5.99,overcast clouds
5,Ushuaia,AR,-54.8000,-68.3000,49.66,62,75,21.85,moderate rain
6,Yulara,AU,-25.2406,130.9889,60.64,77,0,6.91,clear sky
7,Punta Arenas,CL,-53.1500,-70.9167,51.91,50,40,32.21,scattered clouds


In [9]:
# Create the output file (CSV).
output_data_file = "../Weather_Database/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")